# Sparse and redundant representations - notes 

## Intro

The field deals with problems stemming from underdetermined linear systems ($x \in \mathbb{R}^n$, $y \in \mathbb{R}^m$ where $m < n$)

$$Ax = b$$

The problem of finding such $x$'s is ill, posed, but there are situations where finding 

$$x^{*} = \underset{Ax = b}{argmin}|supp(x)| \tag{P0}$$

Is pretty useful.

$supp(x)$ is also denoted $\|x\|_0$ (which is abuse of notation, since $\|\cdot\|_0$ is not a norm - it is not homogenous)

#### Troubles with solving  $P_0$

In general the problem of finding such $x^{*}$ is known to be NP-complete.

Because of this approximations are used in practice.

There are 2 different approaches to approximately solve $P_0$:

- Greedy methods
- Relaxation methods

## Guarantees

Though solving $P_0$ exactly is tough, there are conditional guarantees for solutions of corresponding linear system.

Define $spark(A)$ as the smallest $s$ such that exist columns $(A_{i_j})_{j < s}$ that are linearly dependent (this is actually $\|x^{*}\|_0$ for $P_0$ with $b = 0$). In other words, it's smallest $\|x\|_0$ for $x \in ker(A)$ ($ker(A) = \{x \in \mathbb{R}^n : Ax = 0\}$).

If $x$ is a solution with $\|x\|_0 < \frac{spark(A)}{2}$ then it is the sparsest solution.

Proof of the claim above: 
If $x,y$ are solutions and $max\{\|x\|_0, \|y\|_0\} < \frac{spark(A)}{2}$,

then $0 = Ax - Ay = A(x-y)$, 

so $\|x - y\|_0 \leq \|x\|_0 + \|y\|_0  < spark(A)$, a contradiction.

#### Upper bound on spark

Spark is difficult to compute (computing it in general is NP-hard), but there are methods for upper bounding it by something that is easier to compute.

Define *mutual coherence* of column-normalized $A$, 

$$\mu(A) = max_{i \neq j} A^T_i A_j$$

**Theorem**

$spark(A) \leq 1 + \frac{1}{\mu(A)}$

This theorem will use fact that is an easy consequence of 

**Gershgorin's circle theorem**:

Let $A$ be any square matrix. Then its eigenvalues lie in union of disks of the form 
$ D_k 
= \{ x : | x - A_{kk} | \leq \sum_{i \neq k}{|A_{ki}|} \} $

Precisely we'll use the 

**Fact** 

If $A$ is diagonally dominant ($\forall k$ $|A_{kk}| \geq \sum_{i \neq k}{|A_{ki}|}$) then $A$ is positive definite.

Proof of the theorem:

Without loss of generality assume that $A$ is column-normalized.

Let $s = spark(A)$. That means that exists a submatrix $A_S$ where $S$ is some $s$-element subset of column indices, such that $A_S$ is not full rank. 

That means that $det(A_S) = 0$. It is also equivalent that $det(A_S {A_S}^T) = 0$. Since Gram matrix $A_S {A_S}^T$ is positive semidefinite, it means that it is not positive definite. Using above fact it means that it is not diagonally dominant.

Note that diagonal entries of $A_S$ are equal to one. That means that exists $k$ such that

$$1 = A_{kk} \leq \sum_{i \neq k}{|A_{ki}|} \}$$

On the other hand, by definition of mutual coherence,  $$\sum_{i \neq k}{|A_{ki}|} \leq (s-1)\mu({A})$$

So $1 \leq (s-1)\mu({A})$. By rewriting this we obtain $$s \geq 1 + \frac{1}{\mu(A)}$$

## Approximation algorithms

**Greedy methods** start with $x_0 = 0$ and  built up $x_{n+1}$ by enlarging support.

**Relaxation methods** are based on using continuous optimization tools to relaxations of $\|\cdot\|_0$, for example $\|\cdot\|_1$ is convex and can be also tackled with linear programming tools.


### Greedy methods

They are called matching pursuit in general.

* Least Squares Orthogonal Matching Pursuit (LS-OMP)
    - this is essentially stepwise regression
    - enlarge support with entry that minimizes LS error of new support
* OMP
    - enlarge support with entry most correlated with residual
    - refit coefficients with LS
* Matching Pursuit (MP)
    - same as OMP, but without LS step
* Weak MP
    - needs constant $t$
    - find first entry that would minimize error more than $t \|r\|_2$
* Thresholding
    - sort $A^T b$
    - add coefficients according to order
    
The following code implements OMP, MP and WMP in Python (numpy)

In [ ]:
def pursuit(A, b, n_steps, pursuit_type='omp', wmp_t=None):
    def get_next_approximation_mp(**kwargs):
        a = kwargs['a_next']
        best_column_index = kwargs['best_column_index']
        x_approx = kwargs['x_approx']
        x_approx[best_column_index] += a * np.linalg.norm(res)
        return x_approx

    def get_next_approximation_omp(**kwargs):
        support = kwargs['support']
        x = np.zeros(A.shape[1])
        x_star = np.linalg.lstsq(A[:, support], b)[0]
        x[support] = x_star
        return x
    
    def get_best_column_wmp(inner_products, wmp_t):
        assert wmp_t is not None
        for i, p in enumerate(inner_products):
            residual_error_norm = np.linalg.norm(res)
            if p >= wmp_t * residual_error_norm:
                return i
        return i
    
    res = b
    support = []
    x_approx = np.zeros(A.shape[1])
    
    get_best_column = np.argmax
    if pursuit_type == 'omp':
        get_next_approximation = get_next_approximation_omp
    elif pursuit_type == 'mp':
        get_next_approximation = get_next_approximation_mp
    elif pursuit_type == 'wmp':
        get_next_approximation = get_next_approximation_mp
        get_best_column = lambda products: get_best_column_wmp(products, wmp_t)

    for __ in range(n_steps):
        inner_products = A.T @ res
        best_column_index = get_best_column(inner_products)
        a_next = inner_products[best_column_index]
        #inner_products[best_column_index] = 0
        best_column = A[:, best_column_index]
        support.append(best_column_index)
        x_approx = get_next_approximation(
            a_next=a_next,
            best_column_index=best_column_index,
            support=support,
            x_approx=x_approx)
        res = b - A @ x_approx

    return support, res, x_approx

### Relaxation methods

* Gradually smooth $L_0$ norm - approximate it with a sequence of smooth functions for which first (or even second) order methods are feasible
* Convex relaxation - replace $L_0$ with $L_1$. This is a convex problem, and since the constraints are linear, it can be solved with linear programming.

## Sources

[Michael Elad](https://elad.cs.technion.ac.il) is the go-to person for this kind of things. His team prepared the aforementioned edX Course, and also he's a author/coauthor of these resources:

* [Five Lectures on Sparse and Redundant Representations Modelling of Images](https://elad.cs.technion.ac.il/wp-content/uploads/2018/02/PCMI2010-Elad.pdf) - highly accessible short booklet
* [Sparse and Redundant Representations book](https://www.springer.com/gp/book/9781441970107) - more detailed tha previous report
* [From Sparse Solutions of Systems of Equations to Sparse Modeling of Signals and Images](http://www.geintra-uah.org/system/files/review_paper_siam_review.pdf) - more dense and advanced review